In [1]:
import pandas as pd
from numpy import dot
from numpy.linalg import norm
import numpy as np
import glob


locationSTRING = '지번코드'


# 시간대별 csv파일들 열기
## 추천에 사용할 시간대별 데이터 파일 오픈.
def openCsv1():
    path=r'C:\Users\26060\swa\item'
    files = glob.glob(path+'/*.csv')
    # 파일을 옛날것부터 정렬해준다
    # 파일의 이름은 미리 정리해주었다
    # ex) 201503, 201502,
    files.sort()
    
    # csv 리스트 생성
    csvList = list()
    for fileName in files:
        item=pd.read_csv(fileName, encoding='euc-kr')
        # 지번번호가 있는 csv파일만 활용하자
        if locationSTRING in item.columns:
            csvList.append(item)
    return csvList


csvList1 = openCsv1()
print(len(csvList1))
for csv in csvList1:
    print(locationSTRING in csv.columns)
    
    

# 성연령별 csv파일들 열기
def openCsv2():
    path=r'C:\Users\26060\swa\itemAge'
    files = glob.glob(path+'/*.csv')

    files.sort()
    
    # csv 리스트 생성
    csvList = list()
    for fileName in files:
        item=pd.read_csv(fileName)

        if locationSTRING in item.columns:
            csvList.append(item)
    return csvList

csvList2=openCsv2()
print(len(csvList2))
for csv in csvList2:
    print(locationSTRING in csv.columns)

    
    # 결과측정에 사용할 시간대별 데이터 파일 오픈.
def openCsvTest():
    path=r'C:\Users\26060\swa\itemtest'
    files = glob.glob(path+'/*.csv')

    files.sort()
    
    # csv 리스트 생성
    csvList = list()
    for fileName in files:
        item=pd.read_csv(fileName, encoding='euc-kr')

        if locationSTRING in item.columns:
            csvList.append(item)
    return csvList

csvListTest = openCsvTest()
print(len(csvListTest))
for csv in csvListTest:
    print(locationSTRING in csv.columns)

11
True
True
True
True
True
True
True
True
True
True
True
5
True
True
True
True
True
5
True
True
True
True
True


In [2]:
# 추천엔진을 위해 미리 정의하는 함수들

# 시간을 두자리 숫자 string으로 변환
def timeTo2time(time):
    if time >=24:
        time = time-24
        
    if time>=0 and time <=9:
        return '0'+str(time)
    else:
        return str(time)


# time부터 long시간동안 사람이 많은 지역번호 뽑기
# 예를 들어 여러 해에 걸친 각각의 csv 파일들에서
# 9시부터 4시간동안 유동인구가 많은 지역 추출
# [set, set, set, set,...]
# [1월탑100 , 2월탑100, 3월탑100, .....]
def manyPeople(csvs, time, long):
    # 어떠한 시간대에 유동인구가 많은 상위 n개의 지역 정하기
    n=100
    
    # time부터 long시간동안 유동인구가 많은 n개의 지역을 뽑을 것이다.
    timeList = []
    for i in range(time, time + long):
        s=timeTo2time(i)+'시~'+timeTo2time(i+1)+'시 유동인구'
        timeList.append(s)
        print(s)
    
    sliceDataList = []
    i=0
    for aCsv in csvs:
        
        # 한 csv 파일을 여러 시간에 대해 정렬 한후
        # 상위 n개로 자르고 지역번호만 추출해서 저장
        sliceDataS = set()
        for aTime in timeList:
            i+=1
            # 한 csv 상위 n개 자르기
            aCsv.sort_values(by=[aTime],axis=0,ascending=False, inplace=True)
            sliceDataTemp = aCsv.iloc[0:n,:]
            sliceDataTemp = set( sliceDataTemp[locationSTRING].tolist() )
            sliceDataS = sliceDataS | sliceDataTemp
        
        sliceDataList.append(sliceDataS)
        
    return sliceDataList

# 전체 기간동안 꾸준히 유동인구가 많은 지역
def long_many_people(sliceDataList):
    n = len(sliceDataList)
    i=0
    
    if n == 1:
        return sliceDataList[0]
    
    FSset = sliceDataList[0] & sliceDataList[1]
    if n == 2:
        return FSset
    
    for aSet in sliceDataList:
        FSset = FSset & aSet
    
    
    return FSset

# 여러개 집합들 합집합 하는 함수 만들기
def sets_union(listOfsets):
    ret = set()
    for aSet in listOfsets:
        ret = ret | aSet
    return ret

# 요즘 유동인구가 많아진 지역 
def hot_location(listOfsets):
    # 전체 - 앞구간
    # 구간을 어떻게 정할까
    import math
    N = len(listOfsets)
    b = math.ceil(N * 3 / 5)
    
    # 전체 합집합
    A = sets_union( listOfsets[:] )
    # 앞부분 합집합
    B = sets_union( listOfsets[:b] )
    
    ret = A - B
    return ret



# 추천엔진에서 추천해줬으면 하는 지역번호들 추출.
# 과거에 비해 유동인구가 늘어난 지역
def testSet(csvListA,csvListB):
    # 전체 집합 세팅
    tempA = manyPeople(csvListA,9,4)
    tempB = manyPeople(csvListB,9,4)
    tempA = sets_union(tempA)
    tempB = sets_union(tempB)
    Total_set = tempA | tempB
    Front_set = tempA
    return Total_set - Front_set



In [3]:
# 유사도 분석시, 그외 모든 활용에서
# 시간대별 테스트 csvs, 시간대별 csvs, 성연령별 csvs에
# 모두 공통되는 지역번호만 사용하자.
# 공통되는 지역번호들의 집합 생성.

##################################################
# domain에 속하는 지역번호만 빼고 다 csv에서 지울것임
domain = set()

# csvLIst1(시간대별 유동인구의 교집합)
AAA = set(csvList1[0][locationSTRING].tolist())
    #print(len(AAA))
for aaa in csvList1:
    AAA = AAA & set(aaa[locationSTRING].tolist())
    #print(len(AAA))

# csvLIst1(성연령별 유동인구의 교집합)
BBB = set(csvList2[0][locationSTRING].tolist())
    #print(len(BBB))
for bbb in csvList2:
    BBB = BBB & set(bbb[locationSTRING].tolist())
    #print(len(BBB))

# csvListTest(테스트용 시간대별 유동인구 교집합)
CCC = set(csvListTest[0][locationSTRING].tolist())
    #print(len(CCC))
for ccc in csvListTest:
    CCC = CCC & set(ccc[locationSTRING].tolist())
    #print(len(CCC))

domain=AAA&BBB&CCC
len(domain)


# 공통된 지역번호를 가진 아이들만 사용하자.
for i in range(len(csvList1)):
    temp = csvList1[i]
    temp = temp[temp[locationSTRING].isin(domain)]
    temp = temp.reset_index(drop=True)
    csvList1[i]=temp
    print(len(csvList1[i]))

for i in range(len(csvList2)):
    temp = csvList2[i]
    temp = temp[temp[locationSTRING].isin(domain)]
    temp = temp.reset_index(drop=True)
    csvList2[i]=temp
    print(len(csvList2[i]))
    
for i in range(len(csvListTest)):
    temp = csvListTest[i]
    temp = temp[temp[locationSTRING].isin(domain)]
    temp = temp.reset_index(drop=True)
    csvListTest[i]=temp
    print(len(csvListTest[i]))

35417
35417
35417
35417
35417
35417
35417
35417
35417
35417
35417
35417
35417
35417
35417
35417
35417
35417
35417
35417
35417


In [4]:
# hotLocNum에서 뽑은 벡터들을
# 성연령별에서 비교하지 않고
# 최근 5개 시간대별 csv에서 유사도 분석후 추천

csvList2 = []    # 초기화
csvList2 = csvList1[len(csvList1)-5 : ]
len(csvList2)

# 유사도분석을위하여 필요한부분만냄겨두자
for i in range(len(csvList2)):
    if '지번명' in csvList2[i].columns:
        csvList2[i].drop(['지번명'],axis='columns', inplace = True)
    if '기준년월' in csvList2[i].columns:
        csvList2[i].drop(['기준년월'],axis='columns', inplace = True)
        

# 추천해줬으면 하는 test값 추출
testLocNum = testSet(csvList1,csvListTest)
print(len(testLocNum))
len(set(csvList2[4][locationSTRING].tolist()) & testLocNum)

09시~10시 유동인구
10시~11시 유동인구
11시~12시 유동인구
12시~13시 유동인구
09시~10시 유동인구
10시~11시 유동인구
11시~12시 유동인구
12시~13시 유동인구
29


29

In [9]:

ageCsvIndexList = []
# 사용할 벡터부분만 저장
for aCsv in csvList2:
    temp = aCsv.set_index(locationSTRING).iloc[:,24:]
    ageCsvIndexList.append(temp)
    


# 아래 두개 중 하나로 hotLocNum을 설정한다.

# hot 지역으로 유사도 분석하여 추천하여보자
item = manyPeople(csvList1,9,4)
hotLocNum = list(hot_location(item))
print(len(hotLocNum))
print(hotLocNum)

# 항상 유동인구가 많은 지역으로 유사도 분석하여 추천하여보자
#item = manyPeople(csvList1,9,4)
#hotLocNum = list(long_many_people(item))
#print(len(hotLocNum))
#print(hotLocNum)

09시~10시 유동인구
10시~11시 유동인구
11시~12시 유동인구
12시~13시 유동인구
18
[5011033021126560000, 5013032023200180001, 5011010400111760002, 5013025921101140003, 5011011100110020001, 5013032024201580000, 5011025929201370031, 5011010300113150000, 5011025321125350000, 5011025929201190000, 5011013700103010000, 5011010700110470001, 5011013700102830003, 5011013700102910038, 5013011900130390001, 5013025030200010010, 5011013700102910134, 5011013700102520022]


In [6]:
# 코사인 유사도 함수 정의
def cos_sim(A, B):
    t = np.array(B.shape)
    n = t[0]
    B =  B.reshape(n,1)
    ret = dot(A,B)/(norm(A)*norm(B))
    
    return ret[0]

# 피어쓴 유사도를 위해 각 벡터에
# 평균을 마이너스를 해주는 함수를 만들어주자
# n*1 벡터를..
def minusMeanVector(vec):
    t = np.array(vec.shape)
    n = t[0]
    mean = np.mean(vec)
    
    for i in range(0,n):
        vec[i] -= mean
    return vec

In [7]:
ageCsvIndexList#시간대별csv리스트
hotLocNum     #지역번호 리스트   

import math

# 평가
Eval1 = []
Eval2 = []
Eval3 = []

def cos_sim_aVec_aIndexCsv(aVec, aIndexCsv):
    print("```````")
    # 평균빼준후 전달했지만 그냥 한번 더 해준다
    doc1 = minusMeanVector(aVec)
    
    # csv 매트릭스화
    A = np.array(aIndexCsv)
    
    # 한 csv파일에 있는 모든 지역과 유사도 분석
    t = np.array(A.shape)
    n=t[0]
    m=t[1]
    retDic = {}  #딕셔너리, csv튜플 순서와 유사도 저장
    for i in range(0,n):
        # csv 한 행 벡터 추출
        doc2 = A[i]
        doc2 = minusMeanVector(doc2)
        # 유사도 분석
        cs = cos_sim(doc1, doc2)
        retDic[i]=cs
        #print(cs)
        
    # 이렇게 유사도분석해서 딕셔너리에 저장되어있는 것들을
    # 유사도큰것기준으로정렬
    sss = sorted(retDic, key = lambda k : retDic[k], reverse=True)
    # csv 튜플 순서로 reindex
    aCsv = aIndexCsv.reset_index(drop=False)   
    
    # 유사도가 큰것부터 testset과 비교
    # testset을 모두 추천하는 회수 i가 크면 안좋은것..
    retList =[]
   
    n=0    # 제대로 추천해준 개수
    N = len(testLocNum)    # 전체 정답 개수
    N25 = int(N/4)    # 전체 정답의 25퍼센트
    wholeIter = 0    # 전체를 추천해준데 걸린 누적 iter수
    wholeIter25=0    # 25%를 추천해준데 걸린 누적 iter수
    cnt=[]
    
    k=0 #비교 회수 측정
    locNumList = aCsv[locationSTRING].tolist()    # 실수손실방지
    for i in sss:
        locNum = locNumList[i]
        # 이 지역번호가 우리의 test셋에 존재하면
        if locNum in testLocNum:    
            retList.append(locNum) # 추천 지역번호에 추가.
            # 얼마나 비교해서 test를 찾았는지 기록
            wholeIter = wholeIter + k 
            cnt.append(k)    # 몇번째에 위치하는지 기록
            n = n+1
            
            if n==N25:
                wholeIter25 = wholeIter
            if n == N:
                print('break')
                break
        k=k+1
        
    
    retSet = set(retList)
    
    # 전체 데이터에 대한 등급
    
    a=math.log10(wholeIter)
    print(a)
    # 25% 데이터에 대한 등급
    b = math.log10(wholeIter25)
    print(b)
    # 전체csv에대해 얼마나 빨리 찾았는지.. 0~1
    c = np.mean(cnt) / len(locNumList)
    print(c)
    
    # 기록
    Eval1.append(a)
    Eval2.append(b)
    Eval3.append(c)
    
    
    return retSet

def recommend_hotLocNum_ageCsvIndexList(locNumList,csvList):
    N = len(locNumList)
    
    # 리턴할 집합(추천해줄 지역번호 집합)
    recommendSet = set()
    i=1
    for a in locNumList:
        print('...%d / %d'%(i,N))
        for b in csvList:
            # 핫벡터리스트중 하나가 유사도분석하고자하는
            #csv에 존재한다면, 아닐수가 없다.
            if a in b.index.tolist():
                # 한 지역번호에 해당하는 벡터를 추출
                aVec = b.loc[int(a)].to_numpy()
                aVec = minusMeanVector(aVec)
                # 한 csv와 유사도 분석후 나온 결과를
                #최종 추천에해당하는 집합에 추가
                tempRetSet = cos_sim_aVec_aIndexCsv(aVec,b)
                
                # Mvec,Ncsv에 대한 합집합 추천
                recommendSet = recommendSet | tempRetSet

            else:
                print("error")
        print('...')
        i=i+1
    
    return recommendSet

def sigma(n):
    ret = 0
    for i in range(n+1):
        ret = ret+i
    return ret
sigma(5)

15

In [10]:
ageCsvIndexList#시간대별csv리스트
hotLocNum     #지역번호 리스트   

# 추천 엔진 실행
recommend=recommend_hotLocNum_ageCsvIndexList(hotLocNum, ageCsvIndexList)

...1 / 18
```````
break
5.7709925825273105
4.656174891838548
0.5746227459441355
```````
break
5.744844304402152
4.654869319468523
0.5410464290964888
```````
break
5.769165297258252
4.697726142546344
0.5722101114504724
```````
break
5.768000619046345
4.696067928410897
0.5706776309448122
```````
break
5.735353741534094
4.637139350726339
0.5293512856187317
...
...2 / 18
```````
break
5.7777399235486735
4.653106339924657
0.583619983779463
```````
break
5.781062822951323
4.649461428344476
0.5881025379396023
```````
break
5.781962781491092
4.676016496965091
0.589322485889788
```````
break
5.779701805720802
4.687305960192069
0.5862623929868084
```````
break
5.768493065587071
4.633205779494404
0.5713250893541286
...
...3 / 18
```````
break
5.581623160927494
3.5747255835940734
0.37154668564579835
```````
break
5.655015431346167
3.4387005329007363
0.43995237042799434
```````
break
5.522294226024801
3.330210784571528
0.3241040490004313
```````
break
5.632008593211719
3.3296012483565187
0.41725238

In [11]:
# 평가 값 살펴보기

print(np.mean(Eval1))
print(np.mean(Eval2))
print(np.mean(Eval3))
print()
print(np.max(Eval1))
print(np.max(Eval2))
print(np.max(Eval3))
print()
print(np.min(Eval1))
print(np.min(Eval2))
print(np.min(Eval3))

5.688568420471683
4.387611082472132
0.5003883029407495

5.915550730285269
5.127289496289813
0.8015690886803825

5.35685538958925
3.2132520521963968
0.22143467047287832


In [ ]:
끝